In [1]:
from vnpy.trader.utility import load_json, save_json
# 更新配置文件数据库名称
database_name = "TBDATA.db"

setting = load_json("vt_setting.json")
setting.update({"database.database": database_name})
save_json("vt_setting.json", setting)

In [2]:
from vnpy.trader.engine import MainEngine, EventEngine
from vnpy.app.data_manager.engine import ManagerEngine

manager = ManagerEngine(MainEngine, EventEngine)

提示：当前环境pandas版本为0.25，get_price与get_fundamentals_continuously接口panel参数将固定为False
注意：0.25以上版本pandas不支持panel，如使用该数据结构和相关函数请注意修改
在使用天勤量化之前，默认您已经知晓并同意以下免责条款，如果不同意请立即停止使用：https://www.shinnytech.com/blog/disclaimer/


In [6]:
# 将CSV行情数据导入数据库
manager.import_data_from_csv(
        file_path = "rb888.csv",
        symbol    = "rb888",
        exchange  = "SHFE",
        interval  = "1m",
        datetime_head = "//时间",
        open_head   = "开盘价",
        high_head   = "最高价",
        low_head    = "最低价",
        close_head  = "收盘价",
        volume_head = "成交量",
        open_interest_head = "持仓量",
        datetime_format    = "%Y/%m/%d %H:%M"
    )

(datetime.datetime(2010, 1, 4, 9, 0),
 datetime.datetime(2020, 7, 6, 13, 52),
 759616)

In [16]:
# 将MT5下载的外汇CSV行情数据导入数据库
import pandas as pd
from pytz import timezone
from vnpy.trader.object import BarData
from vnpy.trader.database import database_manager

CHINA_TZ = timezone("Asia/Shanghai")

file_path = "XAGUSD_M1_201809132200_202007271053.csv"
vt_symbol = "XAGUSD.OTC"

data = pd.read_csv(file_path, delimiter="\t")
data["datetime"] = pd.to_datetime(data["<DATE>"] + " " + data["<TIME>"])

bars = []
symbol = vt_symbol.split(".")[0]
exchange = vt_symbol.split(".")[1]

for ix, row in data[:-1].iterrows():

    dt = CHINA_TZ.localize(row.datetime.to_pydatetime().replace(tzinfo=None))

    bar = BarData(
        symbol = symbol,
        exchange = exchange,
        datetime = dt,
        interval = "1m",
        open_price = row["<OPEN>"],
        high_price = row["<HIGH>"],
        low_price = row["<LOW>"],
        close_price = row["<CLOSE>"],
        volume = row["<TICKVOL>"],
        open_interest = 0,
        gateway_name = "DB",
    )

    bars.append(bar)

    if not ix % 10000:
        print(f"{ix}\t{dt}")

# 将数据存入数据库
print("开始保存Bar数据至数据库")
database_manager.save_bar_data(bars)
print("数据保存完毕！")

0	2018-09-13 22:00:00+08:00
10000	2018-09-25 06:26:00+08:00
20000	2018-10-04 15:42:00+08:00
30000	2018-10-16 03:03:00+08:00
40000	2018-10-25 15:50:00+08:00
50000	2018-11-06 06:56:00+08:00
60000	2018-11-16 00:13:00+08:00
70000	2018-11-27 14:21:00+08:00
80000	2018-12-06 21:04:00+08:00
90000	2018-12-18 04:27:00+08:00
100000	2018-12-28 13:53:00+08:00
110000	2019-01-09 20:09:00+08:00
120000	2019-01-21 03:21:00+08:00
130000	2019-01-30 13:37:00+08:00
140000	2019-02-08 19:57:00+08:00
150000	2019-02-20 07:34:00+08:00
160000	2019-03-01 15:30:00+08:00
170000	2019-03-12 20:43:00+08:00
180000	2019-03-22 04:55:00+08:00
190000	2019-06-27 19:11:00+08:00
200000	2019-07-09 06:31:00+08:00
210000	2019-07-18 12:37:00+08:00
220000	2019-07-29 18:32:00+08:00
230000	2019-08-08 01:21:00+08:00
240000	2019-08-19 07:15:00+08:00
250000	2019-08-28 13:17:00+08:00
260000	2019-09-09 00:07:00+08:00
270000	2019-09-18 05:55:00+08:00
280000	2019-09-27 11:40:00+08:00
290000	2019-10-08 17:34:00+08:00
300000	2019-10-18 00:22:

In [4]:
from vnpy.trader.constant import Interval, Exchange

# 获取数据库中所有品种的Bar数据统计信息，包括字段：symbol，exchange，interval，count，start，end
bar_data_statistics = manager.get_bar_data_available()

# 从数据接口下载更新数据库中所有品种的Bar数据至最新
for d in bar_data_statistics:
    num = manager.download_bar_data_from_datasource(d["symbol"], Exchange(d["exchange"]), Interval(d["interval"]), d["end"])
    print(f'{d["symbol"]} \t新下载更新Bar数量：{num}')

auth success 
A9999 新下载更新Bar数量：2
AG9999 新下载更新Bar数量：2
AL9999 新下载更新Bar数量：2
AP9999 新下载更新Bar数量：2
AU9999 新下载更新Bar数量：2
B9999 新下载更新Bar数量：2
BB9999 新下载更新Bar数量：2
BU9999 新下载更新Bar数量：2
C9999 新下载更新Bar数量：2
CF9999 新下载更新Bar数量：2
CJ9999 新下载更新Bar数量：2
CS9999 新下载更新Bar数量：2
CU9999 新下载更新Bar数量：2
EB9999 新下载更新Bar数量：2
EG9999 新下载更新Bar数量：2
FB9999 新下载更新Bar数量：2
FG9999 新下载更新Bar数量：2
FU9999 新下载更新Bar数量：2
HC9999 新下载更新Bar数量：2
I9999 新下载更新Bar数量：2
IC8888 新下载更新Bar数量：2
IC9999 新下载更新Bar数量：2
IF8888 新下载更新Bar数量：2
IF9999 新下载更新Bar数量：2
IH8888 新下载更新Bar数量：2
IH9999 新下载更新Bar数量：2
J9999 新下载更新Bar数量：2
JD9999 新下载更新Bar数量：2
JM9999 新下载更新Bar数量：2
L9999 新下载更新Bar数量：2
M9999 新下载更新Bar数量：2
MA9999 新下载更新Bar数量：2
NI9999 新下载更新Bar数量：2
NR9999 新下载更新Bar数量：2
OI9999 新下载更新Bar数量：2
P9999 新下载更新Bar数量：2
PB9999 新下载更新Bar数量：2
PG9999 新下载更新Bar数量：2
PP9999 新下载更新Bar数量：2
RB9999 新下载更新Bar数量：2
RM9999 新下载更新Bar数量：2
RU9999 新下载更新Bar数量：2
SA9999 新下载更新Bar数量：2
SC9999 新下载更新Bar数量：2
SF9999 新下载更新Bar数量：2
SM9999 新下载更新Bar数量：2
SP9999 新下载更新Bar数量：2
SR9999 新下载更新Bar数量：2
SS9999 新下载更新Bar数量：2
T9999 新下载更新Bar

In [7]:
# 保存数据库中的所有品种的 vt_symbol 至当前工作目录
with open("vt_symbol.txt", "w") as f:
    for d in bar_data_statistics:
        vt_symbol = d["symbol"] + "." + d["exchange"]
        f.write(f"{vt_symbol}\n")